# Initialization

This section initializes GEE, the feature collection for protected land in Ukraine, and the DNBR map function.

In [ ]:
import ee
from ee import geometry as geometry_module

ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=uUsRZEvjj5HOQPlTcVsDSLGk8KkFYVZcrdM9V9XIplw&tc=q_Uj0oUwjBrbmg55dR6NISwwv0ZJ2P3rQR2R8KaJcgo&cc=qa0xPPrd-E2aD641ujsoxqoxLsH23ywVpTIJe22au7w

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXnvW256HM-YwKsDUIiIE1dvm2GhRfcIIgr72QLUoow8Y4TqHTlaw6I

Successfully saved authorization token.


In [ ]:
import geemap
from google.colab import drive
import os
import geemap
import json

drive.mount('/content/drive')

# Getting Ukraine Protected lands shapefile and converting to feature collection
shp2 = 'projects/ee-gkepets/assets/ukraine2'

feature_collection = ee.FeatureCollection(shp2)
rgbVis = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 4000}

Mounted at /content/drive


In [ ]:
# List the names of all the parks
names = feature_collection.aggregate_array('NAME')

names_list = names.getInfo()
display(names_list)

['Kartal Lake',
 'Kugurlui Lake',
 'Shatsk Lakes',
 'Perebrody Peatlands',
 'Somyne Swamps',
 'Syra Pogonia Bog',
 'Somyne Swamps',
 'Archipelago Velyki and Mali Kuchugury',
 'Sim Maiakiv Floodplain',
 'Cheremske Bog',
 'Syra Pogonia Bog',
 'Byle Lake and Koza Berezyna Mire',
 'Dniester-Turunchuk Crossrivers Area',
 'Dnipro River Delta',
 'Kyliiske Mouth',
 'Sasyk Lake',
 'Shagany-Alibei-Burnas Lakes System',
 'Yavorivskyi',
 "Veselivs'kiy",
 "Vichivs'kiy",
 "Vil'khivschins'kiy",
 "Visots'kiy",
 "Volits'kiy",
 "Volits'kiy",
 "Volodimirs'ka dubina",
 "Voloshans'ka dacha",
 'Vorotniv',
 "Vovchans'kiy",
 "Vovchans'kiy",
 "Vtens'kiy",
 "Vutvits'kiy",
 "Yablunivs'kiy",
 "Yaykivs'kiy",
 "Yulivs'ka gora",
 "Yunits'kiy",
 "Zabars'kiy",
 'Zacharovana dolina',
 'Zaplava r.Berda',
 'Zarudka zelena',
 "Zgorans'ki ozera",
 "Zhizhavs'kiy",
 'Zhuravliniy',
 "Zhuravlivs'ka dacha",
 "Zolotins'kiy",
 "Verbivs'kiy",
 "Verbovets'ko-Zalis'kiy",
 "Verkhivs'kiy",
 "Verkhn'oesmans'kiy",
 "Verkhn'osul's'kiy",


# Experimenting with Dynamic World dataset

The dynamic world dataset is a land use / land cover dataset. It has following classes:
* water
* trees
* grass
* flooded_vegetation
* crops
* shrub_and_scrub
* built
* bare
* snow_and_ice

Each band has a probability for the likelihood that a given pixel is that class. This dataset is based on sentinel-2 data and is made by google.


In [ ]:
START = ee.Date('2021-01-02')
END = ee.Date('2021-12-30')
park_name = 'Danube Biosphere Reserve'
matched_feature = feature_collection.filter(ee.Filter.eq('NAME', park_name))
matched_geometry = matched_feature.geometry()

# get dynamic world data and sentinel 2 data
dw_col = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterBounds(matched_geometry).filterDate(START, END)
s2_col = ee.ImageCollection('COPERNICUS/S2').filterBounds(matched_geometry).filterDate(START, END).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

# Create mode composites of both collections... aka find the most frequently guessed label for each pixel
dw_mean = dw_col.mode().clip(matched_geometry)
s2_mean = s2_col.mode()

CLASS_NAMES = [
    'water', 'trees', 'grass', 'flooded_vegetation',
    'crops', 'shrub_and_scrub', 'built', 'bare', 'snow_and_ice',
]

VIS_PALETTE = [
    '419bdf', '397d49', '88b053', '7a87c6',
    'e49635', 'dfc35a', 'c4281b', 'a59b8f', 'b39fe1',
]

dw_rgb = (
    dw_mean.select('label')
    .visualize(min=0, max=8, palette=VIS_PALETTE)
    .divide(255)
)

m = geemap.Map()
m.centerObject(matched_geometry, 12)
m.add_layer(
    s2_mean,
    {'min': 0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']},
    'Sentinel-2 Mean Composite',
)
m.add_layer(
    dw_rgb,
    {'min': 0, 'max': 1.0},
    'Dynamic World V1 - Mean Label Visualization',
)
m

Map(center=[45.43184420158073, 29.65516474353655], controls=(WidgetControl(options=['position', 'transparent_b…

In [ ]:
# This function returns a mask for the desired labels from the DW dataset (see the DW section)

def getDWMask(desired_classes, START, END, matched_feature):
  matched_geometry = matched_feature.geometry()

  # get dynamic world data and sentinel 2 data
  dw_col = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterBounds(matched_geometry).filterDate(START, END)
  s2_col = ee.ImageCollection('COPERNICUS/S2').filterBounds(matched_geometry).filterDate(START, END).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

  # Create a mode composite of DW... aka find the most frequently guessed label for each pixel
  dw_mode = dw_col.mode().clip(matched_geometry)
  s2_mean = s2_col.mean()

  CLASS_NAMES = [
      'water', 'trees', 'grass', 'flooded_vegetation',
      'crops', 'shrub_and_scrub', 'built', 'bare', 'snow_and_ice',
  ]

  VIS_PALETTE = [
      '419bdf', '397d49', '88b053', '7a87c6',
      'e49635', 'dfc35a', 'c4281b', 'a59b8f', 'b39fe1',
  ]

  dw_rgb = (
      dw_mode.select('label')
      .visualize(min=0, max=8, palette=VIS_PALETTE)
      .divide(255)
  )

  m = geemap.Map(height='1000px', width='1000px')
  m.centerObject(matched_geometry, 12)
  m.add_layer(
      s2_mean,
      {'min': 0, 'max': 4000, 'bands': ['B4', 'B3', 'B2']},
      'Sentinel-2 Mode Composite',
  )
  m.add_layer(
      dw_rgb,
      {'min': 0, 'max': 1.0},
      'Dynamic World V1 - Mode Label Visualization',
  )

  mask = ee.Image(0)

  for class_index in desired_classes:
      mask = mask.Or(dw_mode.select('label').eq(class_index))

  return mask, dw_col, m

# Getting dNBR image and Calculating Acreage for each Burn Severity

In [ ]:
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import requests
import shutil

legend_dict = {
    "Unburned": "#047a04",
    "Low Severity": "#e1f011",
    "Moderate/Low Severity": "#f0a911",
    "Moderate/High Severity": "#f04511",
    "High Severity": "#8302bf"
}

# Function to mask unwanted pixels in images
def mask_clouds(image):
  mask = image.select('SCL').eq(11) \
  .Or(image.select('SCL').eq(9)) \
  .Or(image.select('SCL').eq(8)) \
  .Or(image.select('SCL').eq(3)) \
  .Or(image.select('SCL').eq(6)) \
  .Or(image.select('SCL').eq(10))

  return image.updateMask(mask.Not())

# Function to save image as png
def saveImage(name, img, geometry, vis={'min': 0, 'max': 255}, start_date2=''):
  url = img.getThumbURL({
    'region': geometry,
    'dimensions': 1024,
    'format': 'png',
    'crs': 'EPSG:3857',
    'min': vis['min'],
    'max': vis['max']
    })

  # Handle downloading the actual pixels.
  r = requests.get(url, stream=True)
  if r.status_code != 200:
    raise r.raise_for_status()

  filename = name + "_" + start_date2 + '.png'
  with open(filename, 'wb') as out_file:
    shutil.copyfileobj(r.raw, out_file)

# Function to get dNBR image
def generate_dnbr_map(start_date1, end_date1, start_date2, end_date2, feature_collection, name, save_images = False, sentinelClip = None, calculate_acreage=False):
  matched_feature = feature_collection.filter(ee.Filter.eq('NAME', name))

  # Getting collection from period before war
  sentinel_images_before = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
  .filterBounds(matched_feature.geometry()) \
  .filterDate(ee.Date(start_date1), ee.Date(end_date1))

  # Getting collection from period during war
  sentinel_images_after = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
      .filterBounds(matched_feature.geometry()) \
      .filterDate(ee.Date(start_date2), ee.Date(end_date2))

  # Maksing cloud pixels
  sentinel_images_before = sentinel_images_before.map(mask_clouds)
  sentinel_images_after = sentinel_images_after.map(mask_clouds)

  sentinel_images_before_mean_comp = sentinel_images_before.mean()
  sentinel_images_after_mean_comp = sentinel_images_after.mean()

  if(sentinelClip != None):
    sentinel_images_before_mean_comp = sentinel_images_before_mean_comp.clip(sentinelClip.geometry())
    sentinel_images_after_mean_comp = sentinel_images_after_mean_comp.clip(sentinelClip.geometry())

  # Calulating NBR at each pixel for an image
  def calc_NBR(image):
    nbr = image.normalizedDifference(['B8', 'B11']).rename('NBR')
    return image.addBands(nbr)

  sentinel_images_before = sentinel_images_before.map(calc_NBR)
  sentinel_images_after = sentinel_images_after.map(calc_NBR)

  # Getting median NBR composite for before and during collections
  mean_nbr_before = sentinel_images_before.select('NBR').median().clip(matched_feature.geometry())
  mean_nbr_after = sentinel_images_after.select('NBR').median().clip(matched_feature.geometry())

  # Get DW Mask
  mask, dw_col, m = getDWMask([1, 2, 5], start_date1, end_date2, matched_feature)
  nbr1 = mean_nbr_before.updateMask(mask)
  nbr2 = mean_nbr_after.updateMask(mask)

  m.addLayer(sentinel_images_before, {}, 'cloud mask')

  dnbr = mean_nbr_before.subtract(mean_nbr_after).updateMask(mask)

  # Calculating Acreage for each category
  if calculate_acreage:
    value_ranges = [(-2, 0.099, 0), (0.099, 0.269, 1), (0.269, 0.439, 2), (0.439, 0.659, 3), (0.659, 2, 4)]

    new_img = dnbr.select('NBR')

    for min_val, max_val, new_val in value_ranges:
      mask = dnbr.gt(min_val).And(dnbr.lte(max_val))
      new_img = new_img.where(mask, new_val)
    burn_level_img = new_img.rename('burn_level')

    area_dict_final = {
      '0.0': 0,
      '1.0': 0,
      '2.0': 0,
      '3.0': 0,
      '4.0': 0
    }

    area_info = burn_level_img.reduceRegion(
        reducer = ee.Reducer.frequencyHistogram(),
        geometry = matched_feature.geometry(),
        scale = 10,
        maxPixels=1e13
    )

    area_dict = area_info.getInfo()['burn_level']
    total_acreage = 0


    for level in area_dict:
      area_dict_final[level] = area_dict[level] * 0.0247105
      total_acreage += area_dict[level] * 0.0247105


    area_percentage_dict = {}

    for level in area_dict:
      area_percentage_dict[level] = (area_dict[level] * 0.0247105) / total_acreage * 100.0

    area_legend = {
      str(round(area_dict_final['0.0'],2)) + " (" + str(round(area_percentage_dict['0.0'],2)) + "%)": "#047a04",
      str(round(area_dict_final['1.0'],2)) + " (" + str(round(area_percentage_dict['1.0'],2)) + "%)": "#e1f011",
      str(round(area_dict_final['2.0'],2)) + " (" + str(round(area_percentage_dict['2.0'],2)) + "%)": "#f0a911",
      str(round(area_dict_final['3.0'],2)) + " (" + str(round(area_percentage_dict['3.0'],2)) + "%)": "#f04511",
      str(round(area_dict_final['4.0'],2)) + " (" + str(round(area_percentage_dict['4.0'],2)) + "%)": "#8302bf"
    }

    m.add_legend(title='Total Acreage', legend_dict=area_legend)


  # Setting colors based on burn severity levels according to USGS
  sld_intervals = """
    <RasterSymbolizer>
      <ColorMap type="intervals" extended="false" >
        <ColorMapEntry color="#047a04" quantity="0.079" label="unburned" />
        <ColorMapEntry color="#e1f011" quantity="0.269" label="low" />
        <ColorMapEntry color="#f0a911" quantity="0.439" label="mod low" />
        <ColorMapEntry color="#f04511" quantity="0.659" label="mod high" />
        <ColorMapEntry color="#8302bf" quantity="1.3" label="high" />
      </ColorMap>
    </RasterSymbolizer>
    """

  new_dnbr = dnbr.sldStyle(sld_intervals)

  # Saving dNBR image as png
  if(save_images):
    saveImage(name, new_dnbr, matched_feature.geometry())
    if(sentinelClip != None):
      saveImage(name + 'before', sentinel_images_before_mean_comp.select(['B4', 'B3', 'B2']), sentinelClip.geometry(), vis=rgbVis)
      saveImage(name + 'after', sentinel_images_after_mean_comp.select(['B4', 'B3', 'B2']), sentinelClip.geometry(), vis=rgbVis)

  m.add_legend(title='Legend', legend_dict=legend_dict)
  # nbr1 = nbr1.sldStyle(sld_intervals)

  m.addLayer(nbr1, {}, 'NBR1')
  m.addLayer(nbr2, {}, 'NBR2')

  return new_dnbr, sentinel_images_before_mean_comp, sentinel_images_after_mean_comp, m

# Testing on various protected lands that were known to be effected by the war

##Holy Mountains National Park (Svyaty Gory)

In [ ]:
park_name = 'Svyaty Gory'
matched_feature = feature_collection.filter(ee.Filter.eq('NAME', park_name))

img, beforeImg, afterImg, m = generate_dnbr_map('2020-07-01', '2021-07-01', '2022-07-01', '2023-07-01', feature_collection, park_name, save_images = False, calculate_acreage=True)
m.addLayer(beforeImg, rgbVis, 'Before Image')
m.addLayer(afterImg, rgbVis, 'After Image')
m.addLayer(img, {}, 'Mean NBR DNBR mask')
m.addLayer(matched_feature.geometry(), {}, 'park')

m

Map(center=[48.98614416464036, 37.7240556981502], controls=(WidgetControl(options=['position', 'transparent_bg…

##Black Sea Biosphere Reserve

In [ ]:
park_name = 'Black Sea Biosphere Reserve'
matched_feature = feature_collection.filter(ee.Filter.eq('NAME', park_name))

img, beforeImg, afterImg, m = generate_dnbr_map('2021-06-01', '2021-08-01', '2023-06-01', '2023-08-01', feature_collection, park_name, save_images = False, calculate_acreage=True)
m.addLayer(beforeImg, rgbVis, 'Before Image')
m.addLayer(afterImg, rgbVis, 'After Image')
m.addLayer(img, {}, 'Mean NBR DNBR mask')

m

Map(center=[46.30385216535182, 31.875938418575952], controls=(WidgetControl(options=['position', 'transparent_…

## Danube Biosphere Reserve

In [ ]:

park_name = 'Danube Biosphere Reserve'
matched_feature = feature_collection.filter(ee.Filter.eq('NAME', park_name))
img, beforeImg, afterImg = generate_dnbr_map('2020-08-01', '2020-09-01', '2023-09-01', '2023-10-01', feature_collection, park_name, save_images = False)

Map = geemap.Map(toolbar_ctrl=True, layer_ctrl=True)
Map.addLayer(img, {}, 'Mean NBR DNBR')
rgbVis = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 4000}

Map.addLayer(beforeImg, rgbVis, 'Mean Before Composite')
Map.addLayer(afterImg, rgbVis, 'Mean After Composite')
Map.add_legend(title = "dNBR Level", legend_dict = legend_dict)

Map.centerObject(matched_feature, 10)
Map

12

12

Map(center=[45.43184420158073, 29.65516474353655], controls=(WidgetControl(options=['position', 'transparent_b…

# Function to create timelapse of dNBR for each month in a a period of time during war

In [ ]:
from datetime import datetime, timedelta

def get_time_lapse_collection(start_date, end_date, old_year, feature_collection, name, save_images = False):

  ## Getting dates every 30 days
  date_format = "%Y-%m-%d"
  start_date = datetime.strptime(start_date, date_format)
  end_date = datetime.strptime(end_date, date_format)
  current_date = start_date
  current_old_date = current_date.replace(year = old_year)

  dates_list = []
  old_dates_list = []

  while current_date <= end_date:

    dates_list.append(current_date.strftime(date_format))
    old_dates_list.append(current_old_date.strftime(date_format))

    # Move to the first day of the next month
    current_date = current_date.replace(day=1) + timedelta(days=32)
    current_date = current_date.replace(day=1)
    current_old_date = current_date.replace(year = old_year)

  print(dates_list)
  print(old_dates_list)

  collection = ee.ImageCollection([])

  ## Looping through each month and getting dNBR
  for i in range(len(dates_list)-1):
    start_date1 = old_dates_list[i]
    end_date1 = old_dates_list[i+1]

    start_date2 = dates_list[i]
    end_date2 = dates_list[i+1]

    if(start_date1[5:7] == '12'):
      end_date1 = str(int(end_date1[:4]) + 1) + end_date1[4:]

    print(start_date1, end_date1, start_date2, end_date2)

    dNBR_img = generate_dnbr_map(start_date1, end_date1, start_date2, end_date2, feature_collection, name, save_images = save_images)
    collection = collection.merge(ee.ImageCollection([dNBR_img]))

  return collection

## Create GIF from all the dNBRs
def time_lapse_gif(collection, roi):

  vid_params = {
    'dimensions': 1024,
    'region': roi,
    'framesPerSecond': 1,
    'crs': 'EPSG:3857',
  }

  vid_url = collection.getVideoThumbURL(vid_params)

  html_code = f"""<img src="{vid_url}" alt="GEE GIF" />"""

  display(HTML(html_code))



old_year = 2020
start_date = '2022-02-01'
end_date = '2023-11-01'
# park_name = 'Svyaty Gory'
# park_name = 'Danube Biosphere Reserve'
park_name = 'Biloberezhzhia Sviatoslava National Nature Park'
matched_feature_geometry = feature_collection.filter(ee.Filter.eq('NAME', park_name)).geometry()


coll = get_time_lapse_collection(start_date, end_date, old_year = old_year, feature_collection = feature_collection, name = park_name, save_images = True)
time_lapse_gif(coll, matched_feature_geometry)



# Experimenting with Sentinel-1 and SAR data for detecting metal

In [ ]:
start_date = '2022-01-01'
end_date = '2022-12-30'

park_name = 'Svyaty Gory'
matched_feature = feature_collection.filter(ee.Filter.eq('NAME', park_name))

# Getting image collection during war of VH band
vh_during_col = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filterBounds(matched_feature.geometry())
    # .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .select('VH')
    .filterDate(ee.Date(start_date), ee.Date(end_date))
    # .map(mask_edge)
)


start_date = '2021-01-01'
end_date = '2021-12-30'

# Getting image collection before war of VH band
vh_before_col = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filterBounds(matched_feature.geometry())
    # .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .select('VH')
    .filterDate(ee.Date(start_date), ee.Date(end_date))
    # .map(mask_edge)
)

img_vh_during_mean = vh_during_col.mean().clip(matched_feature.geometry())
img_vh_before_mean = vh_before_col.mean().clip(matched_feature.geometry())

img_vh_during_max = vh_during_col.max().clip(matched_feature.geometry())
img_vh_before_max = vh_before_col.max().clip(matched_feature.geometry())

mean_comp = img_vh_during_mean.subtract(img_vh_before_mean)
max_comp = img_vh_during_max.subtract(img_vh_before_max)

# Function to normalize VH values between 0 and 1 based on min and max VH
def normalize(image):
    stats = image.reduceRegion(reducer=ee.Reducer.minMax(), geometry=matched_feature.geometry(), scale=30, maxPixels=1e9)
    minVal = ee.Number(stats.get('VH_min'))
    maxVal = ee.Number(stats.get('VH_max'))
    return image.subtract(minVal).divide(maxVal.subtract(minVal))

normalized_mean_comp = normalize(mean_comp)
normalized_max_comp = normalize(max_comp)

# Displaying on map
m = geemap.Map()
m.centerObject(matched_feature, 10)
m.add_layer(normalize(img_vh_during_mean), {'min': 0, 'max': 1}, 'mean1', True)
m.add_layer(normalize(img_vh_before_mean), {'min': 0, 'max': 1}, 'mean2', True)
m.add_layer(normalized_mean_comp, {'min': 0, 'max': 1}, 'mean', True)
m.add_layer(normalized_max_comp, {'min': 0, 'max': 1}, 'max', True)
m

Map(center=[48.98614416464036, 37.7240556981502], controls=(WidgetControl(options=['position', 'transparent_bg…